In [1]:
import pandas as pd
import torch
import re
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import argparse
import matplotlib.pyplot as plt


# setting random seed
seed = 42
torch.manual_seed(seed)  # cpu
torch.cuda.manual_seed_all(seed)  # gpu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def ChatGLM4_9B_output(content,model,tokenizer):
    messages = [
        {
            "role": "system",
            "content": "你是一个专业的医生，请基于诊疗指南，为以下患者提供综合的管理意见:",
        },
        {
            "role": "user",
            "content": content,
        },
    ]
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(
        device
    )
    outputs = model.generate(**inputs, max_length=2000, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

def diabetesPDiagLLM_output(content, model, tokenizer):
    messages = [
        {
            "role": "system",
            "content": "你是一个专业的医生，请基于诊疗指南，为以下患者提供综合的管理意见:",
        },
        {
            "role": "user",
            "content": content,
        },
    ]

    inputs = tokenizer.apply_chat_template(messages,
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

    inputs = inputs.to(device)
    
    
    gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [2]:

#model_path = args.path
model_path = "../../../autodl-tmp/DiabetesPDiagLLM"
args_model = "DiabetesPDiagLLM" 
if args_model == "DiabetesPDiagLLM":
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    tokenizer.padding_side = "left"
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
        ).to(device).eval()
    model_output = diabetesPDiagLLM_output

if args_model == "ChatGLM-4-9B":
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    # tokenizer.padding_side = "left"
    # tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    model = AutoModel.from_pretrained(model_path, trust_remote_code=True).to(device)
    
    
    model_output = ChatGLM4_9B_output


model = model.eval()
scores = {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Processing src/eval/data/mcq.csv ...


FileNotFoundError: [Errno 2] No such file or directory: 'src/eval/data/mcq.csv'

In [8]:
mcq_file = "data/mcq.csv"

file_name = mcq_file.split("/")[-1]
print(f"Processing {mcq_file} ...")
df = pd.read_csv(mcq_file)
merged_df = pd.DataFrame(columns=["question", "response", "answer"])
prompt = "\n请根据题干和选项，给出唯一的最佳答案。输出内容仅为选项英文字母，不要输出任何其他内容。不要输出汉字。"



Processing data/mcq.csv ...


In [9]:
for index, row in tqdm(df.iterrows()):
        output = {}
        response = model_output(row["question"] + prompt, model, tokenizer)
        output["question"] = [row["question"]]
        output["response"] = [response]
        output["answer"] = row["answer"]
        df_output = pd.DataFrame(output)
        merged_df = pd.concat([merged_df, df_output])
        

312it [02:24,  2.16it/s]


In [10]:
merged_df

,question,response,answer
0,Question: 主要鉴别皮质醇增高是原发于肾上腺还是继发于垂体及下丘脑的试验为\nOpt...,\nE,E：促肾上腺皮质激素(ACTH)兴奋试验
0,"Question: 强直性脊柱炎血清学检查()为阳性,无血尿酸水平升高\nOption A:...",\nB,B：HLA-B27
0,Question: 尿糖阳性的原因不包括\nOption A: 糖尿病\nOption B:...,\nB,B：尿崩症
0,Question: 下列生物半衰期最长的磺脲类降糖药为\nOption A: 甲苯磺丁脲\n...,\nD,B：格列本脲
0,Question: 肢体外周动脉粥样硬化常以()病变为主\nOption A: 主动脉\nO...,\nE,E：下肢动脉
...,...,...,...
0,"Question：患者,女性,39岁。多次尿糖(+)~(++),OGTT各时间点血糖均在正常...",\nB,D：肾性糖尿病
0,"Question：患者,男性,51岁。肥胖症伴高血压,糖耐量减低,尿皮质醇、17-羟皮质类固...",\nA,E：单纯性肥胖症
0,"Question：患者,女性,62岁。确诊2型糖尿病2年,给予合理饮食和运动治疗并口服二甲双...",\nD,E：加用α-葡萄糖苷酶抑制药
0,"Question：患者,男性,50岁。肥胖,2型糖尿病5年,口服二甲双胍250mg,每天2次...",\nA,B：加用胰岛素制剂


In [15]:
merged_df['responseTrimmed'] = merged_df['response'].apply(lambda x: re.search(r'[ABCDE]', x).group() if re.search(r'[ABCDE]', x) else None)

In [21]:
merged_df['check'] = merged_df.apply(lambda row: 1 if row['responseTrimmed'] == row['answer'][0] else 0, axis=1)

In [20]:
merged_df

,question,response,answer,responseTrimmed,check
0,Question: 主要鉴别皮质醇增高是原发于肾上腺还是继发于垂体及下丘脑的试验为\nOpt...,\nE,E：促肾上腺皮质激素(ACTH)兴奋试验,E,1
0,"Question: 强直性脊柱炎血清学检查()为阳性,无血尿酸水平升高\nOption A:...",\nB,B：HLA-B27,B,1
0,Question: 尿糖阳性的原因不包括\nOption A: 糖尿病\nOption B:...,\nB,B：尿崩症,B,1
0,Question: 下列生物半衰期最长的磺脲类降糖药为\nOption A: 甲苯磺丁脲\n...,\nD,B：格列本脲,D,0
0,Question: 肢体外周动脉粥样硬化常以()病变为主\nOption A: 主动脉\nO...,\nE,E：下肢动脉,E,1
...,...,...,...,...,...
0,"Question：患者,女性,39岁。多次尿糖(+)~(++),OGTT各时间点血糖均在正常...",\nB,D：肾性糖尿病,B,0
0,"Question：患者,男性,51岁。肥胖症伴高血压,糖耐量减低,尿皮质醇、17-羟皮质类固...",\nA,E：单纯性肥胖症,A,0
0,"Question：患者,女性,62岁。确诊2型糖尿病2年,给予合理饮食和运动治疗并口服二甲双...",\nD,E：加用α-葡萄糖苷酶抑制药,D,0
0,"Question：患者,男性,50岁。肥胖,2型糖尿病5年,口服二甲双胍250mg,每天2次...",\nA,B：加用胰岛素制剂,A,0
